In [1]:
import os
import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv("deepseek.env")

# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("DEEPSEEK_API_KEY")
api_base = os.getenv("DEEPSEEK_API_BASE")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base
# 为 DeepSeek 配置（可选）
DEEPSEEK_CONFIG = {
    #"model": "gpt-3.5-turbo",  # 或者使用 DeepSeek 兼容模型
    #"openai_api_key": os.environ["OPENAI_API_KEY"],
    # 如果使用 DeepSeek:
    "model": "deepseek-chat",
    "openai_api_base": os.environ["DEEPSEEK_API_BASE"],
    "openai_api_key": os.environ["DEEPSEEK_API_KEY"]
}

### 使用文件加载专用chains


In [7]:
! pip install numexpr
! pip install langsmith

In [ ]:
from langchain.chains import load_chain

chain = load_chain("lc://chains/llm-math/chain.json")

print(chain.run("2+6等于几?"))

ERROR: Could not find a version that satisfies the requirement langchain_hub (from versions: none)
ERROR: No matching distribution found for langchain_hub
Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'langchain_hub'

In [ ]:
chain = load_chain("lc://chains/hello-world/chain.json")

chain.run("男人")

## CustomChain
当通用链不满足的时候，可以自行构建来实现特定的目的
<hr>

In [8]:
from typing import List, Dict, Any, Optional
from langchain_core.callbacks.manager import (
    CallbackManagerForChainRun
)
from langchain.chains.base import  Chain
from langchain_core.prompts.base import BasePromptTemplate
from langchain_core.language_models import  BaseLanguageModel


In [7]:
class wiki_article_chain(Chain):
    """开发一个wiki文章生成器"""
    prompt:BasePromptTemplate
    llm:BaseLanguageModel
    out_key:str="text"

    @property
    def input_keys(self) -> List[str]:
        """将返回Prompt所需的所有键"""
        return self.prompt.input_variables
    
    @property
    def output_keys(self) -> List[str]:
        """将始终返回text键"""
        return [self.out_key]
    
    def _call(
        self,
        inputs:Dict[str,Any],
        run_manager:Optional[CallbackManagerForChainRun]=None,
    ) -> Dict[str,Any]:
        """运行链"""
        prompt_value = self.prompt.format_prompt(**inputs)
        #print("prompt_value:",prompt_value)
        response = self.llm.generate_prompt(
            [prompt_value],callbacks=run_manager.get_child() if run_manager else None
        )
        #print("response:",response)
        if run_manager:
            run_manager.on_text("wiki article is written")
        return {self.out_key:response.generations[0][0].text}
    
    @property
    def _chain_type(self) -> str:
        """链类型"""
        return "wiki_article_chain"
       


In [ ]:
from langchain_community.chat_models import  ChatOpenAI
from langchain_core.prompts import  PromptTemplate

chain = wiki_article_chain(
    prompt=PromptTemplate(
        template="写一篇关于{topic}的维基百科形式的文章",
        input_variables=["topic"]
    ),
    llm=ChatOpenAI(
        temperature=0
    ),
)

/Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [9]:
result = chain.run({"topic":"降本增效"})
print(result)

/Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


降本增效（英文：Cost reduction and efficiency improvement）是指企业或组织通过优化资源配置和管理，降低成本，提高生产效率和经营效益的一种管理方法。降本增效是企业管理中非常重要的一环，可以帮助企业提高竞争力，实现可持续发展。

降本增效的目标是通过降低生产成本和提高生产效率，实现企业的盈利最大化。在实施降本增效的过程中，企业需要进行全面的成本分析，找出造成成本增加的原因，并采取相应的措施进行改进。这些措施可以包括优化生产流程、提高员工效率、降低原材料成本、减少浪费等。

降本增效的好处是显而易见的。首先，通过降低成本，企业可以提高产品的竞争力，降低价格，吸引更多的消费者。其次，提高生产效率可以缩短生产周期，提高产量，增加企业的盈利空间。最后，降本增效还可以帮助企业建立良好的财务基础，为未来的发展打下坚实的基础。

在实施降本增效的过程中，企业需要注意一些关键点。首先，要建立科学的成本管理体系，确保成本数据的准确性和及时性。其次，要注重员工的培训和激励，提高员工的工作积极性和效率。最后，要不断进行成本分析和效率评估，及时调整管理策略，确保降本增效的效果持续。

总的来说，降本增效是企业管理中非常重要的一环，可以帮助企业提高竞争力，实现可持续发展。通过降低成本和提高效率，企业可以实现盈利最大化，为未来的发展奠定坚实的基础。因此，企业应该重视降本增效工作，不断优化管理，提高效率，实现可持续发展。
